# API Exercises

Here are two exercises that involve working with APIs and dictionaries.

One is using the Open Brewery API found at https://www.openbrewerydb.org/, and the other is using the API for UK Police Data, found at https://data.police.uk/docs/.

You can complete them in either order!

Remember that you can create new cells with esc + a or b

## Breweries

### Q1: Load the first page of results with 50 results per page

In [2]:
import requests
url = 'https://api.openbrewerydb.org/breweries?page=1&per_page=50'
r = requests.get(url)
data = r.json()
data[:3]

[{'id': 2,
  'name': 'Avondale Brewing Co',
  'brewery_type': 'micro',
  'street': '201 41st St S',
  'city': 'Birmingham',
  'state': 'Alabama',
  'postal_code': '35222-1932',
  'country': 'United States',
  'longitude': '-86.774322',
  'latitude': '33.524521',
  'phone': '2057775456',
  'website_url': 'http://www.avondalebrewing.com',
  'updated_at': '2018-08-23T23:19:57.825Z',
  'tag_list': []},
 {'id': 4,
  'name': 'Band of Brothers Brewing Company',
  'brewery_type': 'micro',
  'street': '1605 23rd Ave',
  'city': 'Tuscaloosa',
  'state': 'Alabama',
  'postal_code': '35401-4653',
  'country': 'United States',
  'longitude': '-87.5621551272424',
  'latitude': '33.1984907123707',
  'phone': '2052665137',
  'website_url': 'http://www.bandofbrosbrewing.com',
  'updated_at': '2018-08-23T23:19:59.462Z',
  'tag_list': []},
 {'id': 44,
  'name': 'Trim Tab Brewing',
  'brewery_type': 'micro',
  'street': '2721 5th Ave S',
  'city': 'Birmingham',
  'state': 'Alabama',
  'postal_code': '3523

### Q2: This is only the first 50 results.  Get the next 50 and put them together.

In [3]:
url2 = 'https://api.openbrewerydb.org/breweries?page=2&per_page=50'
r2 = requests.get(url2)
data2 = r2.json()
brews = data + data2
brews
len(brews)

100

### Q3: How many of these 100 breweries in are in Alaska?

In [4]:
count = 0
for brew in brews:
    if brew['state'] == 'Alaska':
        count +=1
count    

3

### Q4: Of these 100 breweries, what are the different unique brewery types?

In [5]:
brewery_types = set([brew['brewery_type'] for brew in brews])
brewery_types

{'brewpub', 'contract', 'micro', 'planning', 'proprietor', 'regional'}

### Q5: What is the cloest brewery to "Devil's Potion Brewing Company LLC" ?
* Hint 1: Use Euclidian distance w/ longitude and latitude (assume longitude and latitude are a Carteisan coordinate system)
* Hint 2: You'll have to ignore the entries with "none" for latitude or longitude

In [6]:
# step 1: find Devils's potion Brewery's coordinates
devinfo = []
for brew in brews:
    if brew['name'] == "Devil's Potion Brewing Company LLC":
        devinfo = brew
devinfo 
lon = float(devinfo['longitude'])
lat = float(devinfo['latitude'])
lat

33.1216751

In [7]:
type(brews[0]['longitude'])

str

In [8]:
# step 2: Dictionary of breweries & distance between Devils'info 

distances = {}
for brew in brews:
    if brew['longitude'] == None:
        pass
    else:
        distance = ( ((float(brew['longitude']) - lon)**2) +
                     ((float(brew['latitude']) - lat)**2)
                   )**0.5
        if distance>0:
            distances[brew['name']] = distance
            
#shortest distance besides itself
min_dist = min(list(distances.values()))
min_dist

closest_brew = [k for k,v in distances.items() if v == min_dist]
closest_brew, min_dist

(['Port Brewing Co / The Lost Abbey'], 0.07051129653338688)

### Q6: Write a function to find the closest brewery to any other given brewery

In [9]:
def distance_brew(name):
    # step 1 - find brewery coordinates with name

    for brew in brews:
        if brew['name'] == name:
            long = float(brew['longitude'])
            lati = float(brew['latitude'])
    
    
    #step 2 - find all distances to the brewery
    
    all_distances = {}
    for brew in brews:
        if brew['longitude'] == None:
            pass
        else:
            distance = ( ((float(brew['longitude']) - long)**2) +
                         ((float(brew['latitude']) - lati)**2)
                       )**0.5
        if distance>0:
            all_distances[brew['name']] = distance
    min_distance = min(list(all_distances.values()))
    closest = [k for k,v in all_distances.items() if v == min_distance]
    return closest, min_distance
    
distance_brew("Port Brewing Co / The Lost Abbey")

(["Devil's Potion Brewing Company LLC"], 0.07051129653338688)

### Q7: How would you get the first 10 pages from this API and put them all together using a for loop?

In [11]:
data = []
for i in range(1,11):
    url = 'https://api.openbrewerydb.org/breweries?page=' + str(i) + 'per_page=50'
    r = requests.get(url)
    data += r.json()
len(data)

200

# Crime in the UK

### We will be analyzing different crimes reported in the UK as provided by https://data.police.uk/docs/

# Exploratory analysis
##### 1. How many total crimes were there at latitude : 52.63902 and -1.131321 on November of 2017.
Use the street level crimes data, the documentation for the API can be found at https://data.police.uk/docs/method/crime-street/

In [85]:
url3 = 'https://data.police.uk/api/crimes-street/all-crime?lat=52.63902&lng=-1.131321&date=2017-11'

r3 = requests.get(url3)
r3.status_code
data3 = r3.json()
len(data3)
data3[0]

{'category': 'anti-social-behaviour',
 'location_type': 'Force',
 'location': {'latitude': '52.635121',
  'street': {'id': 883430, 'name': 'On or near High Street'},
  'longitude': '-1.139077'},
 'context': '',
 'outcome_status': None,
 'persistent_id': '',
 'id': 61227717,
 'location_subtype': '',
 'month': '2017-11'}

#2. We've queried the API once, but it could get annoying to retype the url over and over again, create a function `make_api_request` that enables you to query the API.


 The parameters for the function should be:
* lat (float) : latitude
* lng (float) : longitude
* date (string): Date in the format YYYY-MM
    * default value = `None`
    
And it should return a json object of 

for more information on default values check out http://blog.thedigitalcatonline.com/blog/2015/02/11/default-arguments-in-python/

In [18]:
def make_api_request(lat,lng,date=None):
    if date != None:
        link = 'https://data.police.uk/api/crimes-street/all-crime?lat=' + str(lat) +'&lng=' + str(lng) + '&date=' + date
    else:
        link = 'https://data.police.uk/api/crimes-street/all-crime?lat=' + str(lat) +'&lng=' + str(lng)
    resp = requests.get(link)
    return resp.json()



##### 3. Write a function `categories_of_crime` that will determine the count of each type of crime for a given latitude and longitude. This is labelled as 'category' in the records. Your function should call the `make_api_request` function you created.

The parameters for the function should be:

* lat (float) : latitude
* lng (float) : longitude
* date (str) default = None

The function should return:
* a dictionary with the count of each type of crime



Once you've created the function, try it with these locations
* lat, lng of 51.5017861,-0.1432319   (Buckingham Palace)
* lat, lng of 53.480161, -2.245163     (Manchester)

In [19]:
def categories_of_crime(lat,lng,date=None):
    crimes = make_api_request(lat,lng,date)
    crimes_dict = {}
    for crime in crimes:
        if crime['category'] in crimes_dict:
            crimes_dict[crime['category']] += 1
        else:
            crimes_dict[crime['category']] = 1
    return crimes_dict
            
categories_of_crime(51.5017861,-0.1432319,date=None), categories_of_crime(53.480161,-2.245163,date=None)

({'anti-social-behaviour': 686,
  'bicycle-theft': 30,
  'burglary': 131,
  'criminal-damage-arson': 100,
  'drugs': 90,
  'other-theft': 1249,
  'possession-of-weapons': 27,
  'public-order': 179,
  'robbery': 269,
  'shoplifting': 310,
  'theft-from-the-person': 1229,
  'vehicle-crime': 206,
  'violent-crime': 594,
  'other-crime': 25},
 {'anti-social-behaviour': 473,
  'bicycle-theft': 25,
  'burglary': 78,
  'criminal-damage-arson': 108,
  'drugs': 67,
  'other-theft': 387,
  'possession-of-weapons': 19,
  'public-order': 214,
  'robbery': 108,
  'shoplifting': 161,
  'theft-from-the-person': 456,
  'vehicle-crime': 272,
  'violent-crime': 570,
  'other-crime': 19})

**Bonus**: 
* Write a function that determines the difference between Buckingham Palace and Manchester in terms of the number of crimes in each category.
    * In which category is there the largest absolute difference between the category of crime?
* Create a histogram depiction of the categories of crime

In [20]:
buckingham = (51.5017861,-0.1432319)
edinburgh = (55.948811,-3.197982)
manchester = (53.480161,-2.245163)

In [21]:
def diff(lat,lng,date = None):
    crimes = make_api_request(lat,lng,date)
    category_diff = {}
    for category_diff['category'] 
            category_dict[crime['category']] +=1
        else:
            category_dict[crime['category']] = 1
    return category_diff

In [22]:
buck_categories = categories_of_crime(buckingham[0],buckingham[1])
print('Buckingham Palace crime categories:',buck_categories)

Buckingham Palace crime categories: {'anti-social-behaviour': 686, 'bicycle-theft': 30, 'burglary': 131, 'criminal-damage-arson': 100, 'drugs': 90, 'other-theft': 1249, 'possession-of-weapons': 27, 'public-order': 179, 'robbery': 269, 'shoplifting': 310, 'theft-from-the-person': 1229, 'vehicle-crime': 206, 'violent-crime': 594, 'other-crime': 25}


In [23]:
edin_categories = categories_of_crime(edinburgh[0],edinburgh[1])
print('Edinburgh Castle crime categories',edin_categories)

Edinburgh Castle crime categories {'drugs': 2, 'other-theft': 3, 'shoplifting': 3, 'theft-from-the-person': 1, 'violent-crime': 6, 'other-crime': 1}


In [24]:
manch_categories = categories_of_crime(manchester[0],manchester[1])
print(manch_categories)

{'anti-social-behaviour': 473, 'bicycle-theft': 25, 'burglary': 78, 'criminal-damage-arson': 108, 'drugs': 67, 'other-theft': 387, 'possession-of-weapons': 19, 'public-order': 214, 'robbery': 108, 'shoplifting': 161, 'theft-from-the-person': 456, 'vehicle-crime': 272, 'violent-crime': 570, 'other-crime': 19}


##### 4. Create a function `find_outcome_statuses` that will determine outcome statuses for a given latitude and longitude and date (optional)?
Investigate the data to determine where the outcome statuses are located.

**NOTE**: You'll notice that some of these crimes do not have crime outcomes. Make these into the category of "Not Resolved."

**NOTE 2**: These might take a long time to execute if you do not specify a month

**Bonus**: What is the ratio of crimes investigated to those not investigated? Is it higher near London or Manchester?

In [25]:
from collections import Counter

def find_outcome_statuses(lat,lng,date=None):
    outcome_status = []
    crimes = make_api_request(lat,lng,date)
    for crime in crimes:
        if crime['outcome_status']:
            outcome_status.append(crime['outcome_status']['category'])
        else:
            outcome_status.append('Not Resolved')
    d = Counter(outcome_status)    
    return d


##### 5. Write a function `month_highest_crimes` that will return the month that had the highest number of crimes for a latitude, longitude and a year.

Inputs
* lat (float) : latitude
* lng (float) : longitude
* year (str) : in the format YYYY

Output
* month with highest crime (int)

**Bonus** Make a graph of how the number of crimes changed over time for a year. This will likely require a new function. Is seasonality a factor? Do the type of crimes change over time?

### Bonus Open Ended Questions

1. Take a look at the https://data.police.uk/docs/method/stops-street/ API. Is there a correlation between gender and being stopped and searched? How about race and being stopped and searched?